In [ ]:
import os
import time
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['PYTHONPATH'] = ""
sys.path.remove('/home/jovyan/.imgenv-vasilyev-0/lib/python3.7/site-packages')

In [ ]:
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import random
from pytorch_lightning.callbacks import (EarlyStopping, ModelCheckpoint,
                                         ModelSummary, TQDMProgressBar)
from torch.utils.data import DataLoader

from datasets import (CausalLMDataset, CausalLMPredictionDataset,
                         PaddingCollateFn)
from models import SASRec, GRU4Rec
from modules import SeqRec, SeqRecWithSampling
from postprocess import preds2recs
from split import *


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [ ]:
VAl_USER = 500
SAMPLE_SIZE = 1000

In [ ]:
data = pd.read_parquet('test_zvuk.parquet')
data

,user_id,timestamp,item_id,rating
0,1736489,1673740801516000000,13113,3
1,4976699,1673740803052000000,76029,5
2,4976699,1673740803056000000,105990,1
3,2393293,1673740803556000000,22627,3
4,5477350,1673740803965000000,49458,2
...,...,...,...,...
24458788,3961121,1682478698807000000,95865,24
24458789,9067175,1682478698944000000,3615,1
24458790,4180391,1682478699230000000,1519,1
24458791,2088083,1682478699233000000,71478,1


In [ ]:
train = preprocessing(data, item_min_count=5, min_len=5)

val_user = np.random.choice(train.user_id.unique(), int(VAl_USER))
validation = train[train["user_id"].isin(val_user)]

train = train[~train["user_id"].isin(val_user)]
max_item_id = data.item_id.max()

n-core filtering step 1


In [ ]:
validation

,user_id,timestamp,item_id,rating
3335,9527554,1673744487533000000,49146,1
6763,4720720,1673747174725000000,72577,2
7097,6681161,1673747448732000000,80467,1
7295,6681161,1673747600675000000,12985,1
8064,4015583,1673748163094000000,21736,8
...,...,...,...,...
24458063,2734189,1682478574387000000,59835,1
24458210,6586131,1682478598868000000,30904,1
24458320,4426262,1682478617974000000,44777,1
24458713,2792061,1682478686018000000,92529,2


In [ ]:
sampled_users = np.random.choice(train['user_id'].unique(),
                                         size=SAMPLE_SIZE, replace=False)
filtered_train = train[train["user_id"].isin(sampled_users)]

In [ ]:
filtered_train

,user_id,timestamp,item_id,rating
30,8298504,1673740843131000000,795,2
33,3700084,1673740848874000000,54570,1
165,8298504,1673740963438000000,26135,2
266,3700084,1673741148737000000,8598,1
269,8298504,1673741151521000000,93237,7
...,...,...,...,...
24458020,2232328,1682478565936000000,97753,1
24458032,3552171,1682478567743000000,74586,1
24458177,8698874,1682478593377000000,73109,2
24458214,2941278,1682478599482000000,44971,1


In [ ]:
train_loader, eval_loader = create_dataloaders(filtered_train, validation)
model = create_model(item_count=max_item_id)
start_time = time.time()
trainer, seqrec_module, model = training(model, train_loader, eval_loader)
training_time = time.time() - start_time
print('training_time', training_time)

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

   | Name                              | Type                            | Params
---------------------------------------------------------------------------------------
0  | model                             | SASRec                          | 6.9 M 
1  | model.ite

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/Pembek/seqs/models.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  timeline_mask = torch.tensor(input_ids == 0)
/home/jovyan/Pembek/ap_39/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

training_time 18.341002941131592


In [ ]:
seqrec_module.filter_seen = True

In [ ]:
recs_all = predict(trainer, seqrec_module, data, top_k=10)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/jovyan/Pembek/seqs/models.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  timeline_mask = torch.tensor(input_ids == 0)


recs shape (1180690, 3)


In [ ]:
data.user_id.nunique()

118069

In [ ]:
recs_all.nunique()

user_id        118069
item_id         53222
prediction    1056410
dtype: int64

In [ ]:
recs_all

,user_id,item_id,prediction
0,1000101,44836,0.800927
1,1000101,9667,0.743439
2,1000101,104312,0.725186
3,1000101,82171,0.688246
4,1000101,45728,0.635307
...,...,...,...
1180685,9999817,21778,0.569791
1180686,9999817,30434,0.557868
1180687,9999817,106078,0.554215
1180688,9999817,31894,0.551933


In [ ]:
def create_dataloaders(train, validation):

    train_dataset = CausalLMDataset(train, max_length=128)
    eval_dataset = CausalLMPredictionDataset(
                    validation, max_length=128, validation_mode=True, )

    train_loader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, num_workers=8,
                              collate_fn=PaddingCollateFn())
    eval_loader = DataLoader(eval_dataset, batch_size=256,
                             shuffle=False, num_workers=8,
                             collate_fn=PaddingCollateFn())

    return train_loader, eval_loader


def create_model(item_count):

    model = SASRec(item_num=item_count, hidden_units=64, num_blocks=2,
                   num_heads=2, dropout_rate=0.1)

    return model


def training(model, train_loader, eval_loader):

    seqrec_module = SeqRec(model, lr=0.001, predict_top_k=10, filter_seen=False)

    early_stopping = EarlyStopping(monitor="val_ndcg", mode="max",
                                   patience=5, verbose=False)
    model_summary = ModelSummary(max_depth=4)
    checkpoint = ModelCheckpoint(save_top_k=1, monitor="val_ndcg",
                                 mode="max", save_weights_only=True)
    progress_bar = TQDMProgressBar(refresh_rate=100)
    callbacks=[early_stopping, model_summary, checkpoint, progress_bar]

    trainer = pl.Trainer(callbacks=callbacks, enable_checkpointing=True, max_epochs=100)

    trainer.fit(model=seqrec_module,
            train_dataloaders=train_loader,
            val_dataloaders=eval_loader)

    seqrec_module.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])

    return trainer, seqrec_module, model


def predict(trainer, seqrec_module, data, top_k=10):

    predict_dataset = CausalLMPredictionDataset(
        data, max_length=128)

    predict_loader = DataLoader(
        predict_dataset, shuffle=False,
        collate_fn=PaddingCollateFn(),
        batch_size=256,
        num_workers=8)

    seqrec_module.predict_top_k = top_k
    preds = trainer.predict(model=seqrec_module, dataloaders=predict_loader)

    recs = preds2recs(preds)
    print('recs shape', recs.shape)def create_dataloaders(train, validation):

    train_dataset = CausalLMDataset(train, max_length=128)
    eval_dataset = CausalLMPredictionDataset(
                    validation, max_length=128, validation_mode=True, )

    train_loader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, num_workers=8,
                              collate_fn=PaddingCollateFn())
    eval_loader = DataLoader(eval_dataset, batch_size=256,
                             shuffle=False, num_workers=8,
                             collate_fn=PaddingCollateFn())

    return train_loader, eval_loader


def create_model(item_count):

    model = SASRec(item_num=item_count, hidden_units=64, num_blocks=2,
                   num_heads=2, dropout_rate=0.1)

    return model


def training(model, train_loader, eval_loader):

    seqrec_module = SeqRec(model, lr=0.001, predict_top_k=10, filter_seen=False)

    early_stopping = EarlyStopping(monitor="val_ndcg", mode="max",
                                   patience=5, verbose=False)
    model_summary = ModelSummary(max_depth=4)
    checkpoint = ModelCheckpoint(save_top_k=1, monitor="val_ndcg",
                                 mode="max", save_weights_only=True)
    progress_bar = TQDMProgressBar(refresh_rate=100)
    callbacks=[early_stopping, model_summary, checkpoint, progress_bar]

    trainer = pl.Trainer(callbacks=callbacks, enable_checkpointing=True, max_epochs=100)

    trainer.fit(model=seqrec_module,
            train_dataloaders=train_loader,
            val_dataloaders=eval_loader)

    seqrec_module.load_state_dict(torch.load(checkpoint.best_model_path)['state_dict'])

    return trainer, seqrec_module, model


def predict(trainer, seqrec_module, data, top_k=10):

    predict_dataset = CausalLMPredictionDataset(
        data, max_length=128)

    predict_loader = DataLoader(
        predict_dataset, shuffle=False,
        collate_fn=PaddingCollateFn(),
        batch_size=256,
        num_workers=8)

    seqrec_module.predict_top_k = top_k
    preds = trainer.predict(model=seqrec_module, dataloaders=predict_loader)

    recs = preds2recs(preds)
    print('recs shape', recs.shape)

    return recs


def evaluate(recs, test_last, topk=[10], prefix='test'):

    all_metrics = {}

    for k in topk:
        evaluator = Evaluator(top_k=[k])
        metrics = evaluator.compute_metrics(test_last, recs)
        metrics = {prefix + '_' + key: value for key, value in metrics.items()}
        all_metrics.update(metrics)
    print(all_metrics)
    return all_metrics

    return recs


def evaluate(recs, test_last, topk=[10], prefix='test'):

    all_metrics = {}

    for k in topk:
        evaluator = Evaluator(top_k=[k])
        metrics = evaluator.compute_metrics(test_last, recs)
        metrics = {prefix + '_' + key: value for key, value in metrics.items()}
        all_metrics.update(metrics)
    print(all_metrics)
    return all_metrics